### Hoja de trabajo 2
- Paola de León 20361
- Paola Contreras 20213

In [8]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset


*TASK 1  - Preparación del conjunto de datos*

In [9]:

iris = datasets.load_iris()

X = iris.data  
y = iris.target  

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir los datos en tensores de PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_val:", X_val.shape)
print("y_val:", y_val.shape)

X_train: torch.Size([120, 4])
y_train: torch.Size([120])
X_val: torch.Size([30, 4])
y_val: torch.Size([30])


*Task 2 - Arquitectura modelo*

In [10]:
# Basado en  https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html
class Net(nn.Module):
    
    def __init__(self, entrada, salida, ocultas):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(entrada,ocultas[0])  
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(ocultas[0], ocultas[1])
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(ocultas[1], salida)
        

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

entrada = X_train.shape[1]
salida = 4
ocultas = [64,32]

model = Net(entrada, salida, ocultas)
print(model)

Net(
  (fc1): Linear(in_features=4, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=4, bias=True)
)


*Task 3 - Funciones de Pérdida*

In [11]:
# Crear conjuntos de datos y DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_val, y_val)

batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=batch_size)

In [12]:
# Cross Entropy Loss
lr = 0.01
num_epochs = 100
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)


for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    
    train_loss /= len(train_loader.dataset)
    
    # Modo de evaluación en datos de prueba
    model.eval()
    val_loss = 0.0
    
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
    
    val_loss /= len(val_loader.dataset)
    
    # Imprimir información de entrenamiento y validación
    if (epoch + 1) % 10 == 0:
        print(f'Época [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Value Loss: {val_loss:.4f}')


Época [10/100], Train Loss: 0.3703, Value Loss: 0.3867
Época [20/100], Train Loss: 0.1978, Value Loss: 0.1815
Época [30/100], Train Loss: 0.1638, Value Loss: 0.1296
Época [40/100], Train Loss: 0.1158, Value Loss: 0.1942
Época [50/100], Train Loss: 0.1323, Value Loss: 0.0958
Época [60/100], Train Loss: 0.1000, Value Loss: 0.0897
Época [70/100], Train Loss: 0.1170, Value Loss: 0.0874
Época [80/100], Train Loss: 0.1092, Value Loss: 0.2080
Época [90/100], Train Loss: 0.0950, Value Loss: 0.1579
Época [100/100], Train Loss: 0.0882, Value Loss: 0.2932


In [13]:
#  Mean Squared Error
lr = 0.01
loss_function = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

# Hyperparameters
num_epochs = 100

# Training loop
for epoch in range(num_epochs):
    # Training mode
    model.train()
    
    train_loss = 0.0
    
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, targets.view(-1, 1).float()) 
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    
    train_loss /= len(train_loader.dataset)
    
    # Evaluation mode on test data
    model.eval()
    test_loss = 0.0
    
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = loss_function(outputs, targets.view(-1, 1))  
            test_loss += loss.item() * inputs.size(0)
    
    test_loss /= len(val_loader.dataset)
    
    # Print training and validation information
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Value Loss: {test_loss:.4f}')

Epoch [10/100], Train Loss: 0.0539, Value Loss: 0.0549
Epoch [20/100], Train Loss: 0.0446, Value Loss: 0.0451
Epoch [30/100], Train Loss: 0.0400, Value Loss: 0.0490
Epoch [40/100], Train Loss: 0.0400, Value Loss: 0.0428
Epoch [50/100], Train Loss: 0.0375, Value Loss: 0.0428
Epoch [60/100], Train Loss: 0.0368, Value Loss: 0.0432
Epoch [70/100], Train Loss: 0.0361, Value Loss: 0.0424
Epoch [80/100], Train Loss: 0.0377, Value Loss: 0.0418
Epoch [90/100], Train Loss: 0.0355, Value Loss: 0.0419
Epoch [100/100], Train Loss: 0.0358, Value Loss: 0.0418


In [14]:
#  Huber Loss
lr = 0.01
loss_function = nn.SmoothL1Loss()
optimizer = optim.SGD(model.parameters(), lr=lr)

# Hyperparameters
num_epochs = 100

# Training loop
for epoch in range(num_epochs):
    # Training mode
    model.train()
    
    train_loss = 0.0
    
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, targets.view(-1, 1).float()) 
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    
    train_loss /= len(train_loader.dataset)
    
    # Evaluation mode on test data
    model.eval()
    test_loss = 0.0
    
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = loss_function(outputs, targets.view(-1, 1))  # Ensure targets match output shape
            test_loss += loss.item() * inputs.size(0)
    
    test_loss /= len(val_loader.dataset)
    
    # Print training and validation information
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Value Loss: {test_loss:.4f}')

Epoch [10/100], Train Loss: 0.0175, Value Loss: 0.0236
Epoch [20/100], Train Loss: 0.0176, Value Loss: 0.0208
Epoch [30/100], Train Loss: 0.0173, Value Loss: 0.0245
Epoch [40/100], Train Loss: 0.0177, Value Loss: 0.0225
Epoch [50/100], Train Loss: 0.0172, Value Loss: 0.0219
Epoch [60/100], Train Loss: 0.0170, Value Loss: 0.0222
Epoch [70/100], Train Loss: 0.0175, Value Loss: 0.0210
Epoch [80/100], Train Loss: 0.0169, Value Loss: 0.0208
Epoch [90/100], Train Loss: 0.0169, Value Loss: 0.0207
Epoch [100/100], Train Loss: 0.0170, Value Loss: 0.0211
